In [9]:
import requests
import numpy as np
import random
import time

def download_data(url,filename):    #download file as data
    result = requests.get(url)
    result.raise_for_status()
    with open(filename,'wb') as FILE:
        for chunk in result.iter_content(102400):
            FILE.write(chunk)
            
url01 = 'https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mltech/hw4_nnet_train.dat'
url02 = 'https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mltech/hw4_nnet_test.dat'
download_data(url01, 'hw4_nnet_train.txt')
download_data(url02, 'hw4_nnet_test.txt')
    
    
def readout(filename):
    with open(filename,'r') as FILE:
        n, Dx, Dy = 0 ,[], []
        
        for chunk in FILE:
            X= chunk.split()   #split the line into a list of string
            X= [ float(X[j]) for j in range(len(X)) ]  #convert the string to numbers   
#            X.insert(0,1)                #insert a constant as bias or as threshold
            Dx.append(X[0:len(X)-1])     #gather all data ## index len(X)-1 will not be included here
            Dy.append(X[len(X)-1])       #index len(X)-1 only
            n=n+1
        x = np.array(Dx)  #change list X into array x
        y = np.array(Dy)
#    print(x,n)            
    return x,y,n        


def eout(w,testx,testy):
    err = []
    yhat = [np.dot(w,testx[i]) for i in range(testn)]

    for a,b in zip(yhat,testy):
        if a*b<=0:
            err.append(1)
        else:
            err.append(0)
    eout = sum(err) /testn
    return eout


trainx, trainy, trainn = readout('hw4_nnet_train.txt')
testx, testy, testn    = readout('hw4_nnet_test.txt') 

In [8]:
#11 Back propagation

import random
import numpy as np
import time


def initw(n,m):
#    w = [[random.uniform(-r,r) for i in range(m)] for i in range(n+1)]  #vector W 有M個 --- dim of w1: n+1*m matrix
    w = np.random.rand(n+1, m)
    w = (w-0.5)*2*r
    return np.array(w)

def updatew(w,x,delta):          # Gradient descent: w = w - eta*x(l-1)[i]*delta(l)[j]
    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            w[i,j] = w[i,j]-eta*x[i]*delta[j]
    return w
    
def err(w1,w2,x,y,n):
    x = np.insert(x,0,np.full(n,1),axis=1)
    x1 = np.tanh(w1.T.dot(x.T))
    x1 = np.insert(x1,0,np.full(n,1),axis=0)
    yhat = np.tanh(w2.T.dot(x1))
    er = y*yhat<0
    eout = er.sum()/n
    

    return eout
  

m = [1,6,11,16,21]
t = 500        #run t experiments
T = 50000    #run T iteratios per experiment
eta = 0.1
r = 0.1
d = len(trainx[0])
N = trainn



for M in m:
    start = time.time()   
    nndim = [d,M,1]   #d-M-1
    Err = 0
    n0 = nndim[0]       #numbers of x0
    n1 = nndim[1]       #numbers of hidden neurons
    n2 = nndim[2]       #numbers of output neuron
 
    for times in range(t):
        w1 = initw(nndim[0],nndim[1])  
        w2 = initw(nndim[1],nndim[2])


        for iteration in range(T):
            randomn = random.randint(0,N-1)
            x0 = np.insert(trainx[randomn],0,1)  #dimension of d+1
            x0 = np.reshape(x0,(-1,1))           #One shape dimension can be -1, inferred from the length of the array and remaining dim
            s1 = w1.T.dot(x0)
            x1 = np.tanh(s1)

            x1 = np.insert(x1,0,1,axis=0)  #dimension of M+1
            s2 = w2.T.dot(x1)
            x2 = np.tanh(s2)

            delta2 = (-8)*(trainy[randomn]-np.tanh(s2))/(np.exp(s2)+np.exp(-s2))
            delta1 = delta2*w2[1:]*4/(np.exp(s1)+np.exp(-s1))

            w2 = updatew(w2,x1,delta2)
            w1 = updatew(w1,x0,delta1)

        Err = Err+err(w1,w2,testx,testy,testn)
#        print(Err/(times+1))
        
    Eout = Err/t
    
    end = time.time()
    print('M =',M,'Eout =',Eout,'time consumed:',end-start)

M = 1 Eout = 0.32503200000000027 time consumed: 1691.8870646953583
M = 6 Eout = 0.038711999999999906 time consumed: 3444.949547767639
M = 11 Eout = 0.03963999999999993 time consumed: 5051.830329656601
M = 16 Eout = 0.038711999999999934 time consumed: 6889.550478935242
M = 21 Eout = 0.03799199999999987 time consumed: 9046.050089120865


In [6]:
#12 Back propagation with diff r


#m = [1,6,11,16,21]
M = 3
t = 500        #run t experiments
T = 50000    #run T iteratios per experiment
eta = 0.1
R = np.array([0,0.001,0.1,10,1000])
d = len(trainx[0])
N = trainn



for r in R:
    start = time.time()   
    nndim = [d,M,1]   #d-M-1
    Err = 0
    n0 = nndim[0]       #numbers of x0
    n1 = nndim[1]       #numbers of hidden neurons
    n2 = nndim[2]       #numbers of output neuron
 
    for times in range(t):
        w1 = initw(nndim[0],nndim[1])  
        w2 = initw(nndim[1],nndim[2])


        for iteration in range(T):
            randomn = random.randint(0,N-1)
            x0 = np.insert(trainx[randomn],0,1)  #dimension of d+1
            x0 = np.reshape(x0,(-1,1))           #One shape dimension can be -1, inferred from the length of the array and remaining dim
            s1 = w1.T.dot(x0)
            x1 = np.tanh(s1)

            x1 = np.insert(x1,0,1,axis=0)  #dimension of M+1
            s2 = w2.T.dot(x1)
            x2 = np.tanh(s2)

            delta2 = (-8)*(trainy[randomn]-np.tanh(s2))/(np.exp(s2)+np.exp(-s2))
            delta1 = delta2*w2[1:]*4/(np.exp(s1)+np.exp(-s1))

            w2 = updatew(w2,x1,delta2)
            w1 = updatew(w1,x0,delta1)

        Err = Err+err(w1,w2,testx,testy,testn)
#        print(Err/(times+1))
        
    Eout = Err/t
    
    end = time.time()
    print('M =',M,'r =',r,'eta =',eta,'Eout =',Eout,'time consumed:',end-start)

M = 3 r = 0.0 eta = 0.1 Eout = 0.4914880000000004 time consumed: 2516.1669857501984
M = 3 r = 0.001 eta = 0.1 Eout = 0.03891999999999989 time consumed: 2496.7293293476105
M = 3 r = 0.1 eta = 0.1 Eout = 0.03855199999999991 time consumed: 2531.825089454651
M = 3 r = 10.0 eta = 0.1 Eout = 0.1885519999999999 time consumed: 2547.2259657382965


C:\Users\paes20705\anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in exp
C:\Users\paes20705\anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: overflow encountered in exp


M = 3 r = 1000.0 eta = 0.1 Eout = 0.493976 time consumed: 2681.218108892441


In [7]:
#13 Back propagation with diff eta

#m = [1,6,11,16,21]

t = 500        #run t experiments
T = 50000    #run T iteratios per experiment
M = 3
Eta = [0.001,0.01,0.1,1,10]
r = 0.1
d = len(trainx[0])
N = trainn



for eta in Eta:
    start = time.time()   
    nndim = [d,M,1]   #d-M-1
    Err = 0
    n0 = nndim[0]       #numbers of x0
    n1 = nndim[1]       #numbers of hidden neurons
    n2 = nndim[2]       #numbers of output neuron
 
    for times in range(t):
        w1 = initw(nndim[0],nndim[1])  
        w2 = initw(nndim[1],nndim[2])


        for iteration in range(T):
            randomn = random.randint(0,N-1)
            x0 = np.insert(trainx[randomn],0,1)  #dimension of d+1
            x0 = np.reshape(x0,(-1,1))           #One shape dimension can be -1, inferred from the length of the array and remaining dim
            s1 = w1.T.dot(x0)
            x1 = np.tanh(s1)

            x1 = np.insert(x1,0,1,axis=0)  #dimension of M+1
            s2 = w2.T.dot(x1)
            x2 = np.tanh(s2)

            delta2 = (-8)*(trainy[randomn]-np.tanh(s2))/(np.exp(s2)+np.exp(-s2))
            delta1 = delta2*w2[1:]*4/(np.exp(s1)+np.exp(-s1))

            w2 = updatew(w2,x1,delta2)
            w1 = updatew(w1,x0,delta1)

        Err = Err+err(w1,w2,testx,testy,testn)
#        print(Err/(times+1))
        
    Eout = Err/t
    
    end = time.time()
    print('M =',M,'r =',r,'eta =',eta,'Eout =',Eout,'time consumed:',end-start)

M = 3 r = 0.1 eta = 0.001 Eout = 0.036415999999999886 time consumed: 2627.4103178977966
M = 3 r = 0.1 eta = 0.01 Eout = 0.03495199999999988 time consumed: 2651.647789955139
M = 3 r = 0.1 eta = 0.1 Eout = 0.03839199999999985 time consumed: 3090.625109910965
M = 3 r = 0.1 eta = 1 Eout = 0.49131200000000014 time consumed: 2531.1620848178864
M = 3 r = 0.1 eta = 10 Eout = 0.49652800000000036 time consumed: 2531.8207507133484


In [7]:
#14 Back propagation with DNN

def initw(n,m):
#    w = [[random.uniform(-r,r) for i in range(m)] for i in range(n+1)]  #vector W 有M個 --- dim of w1: n+1*m matrix
    w = np.random.rand(n+1, m)
    w = (w-0.5)*2*r
    return np.array(w)

def updatew(w,x,delta):          # Gradient descent: w = w - eta*x(l-1)[i]*delta(l)[j]
    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            w[i,j] = w[i,j]-eta*x[i]*delta[j]
    return w
def err(w1,w2,w3,x,y,n):
    x = np.insert(x,0,np.full(n,1),axis=1)
    x1 = np.tanh(w1.T.dot(x.T))
    x1 = np.insert(x1,0,np.full(n,1),axis=0)
    x2 = np.tanh(w2.T.dot(x1))
    x2 = np.insert(x2,0,np.full(n,1),axis=0)
    yhat = np.tanh(w3.T.dot(x2))
    er = y*yhat<0
    eout = er.sum()/n
    
    return eout


#m = [1,6,11,16,21]

t = 500        #run t experiments
T = 50000    #run T iteratios per experiment
M = 8
m = 3
eta = 0.01
r = 0.1
d = len(trainx[0])
N = trainn


start = time.time()   
nndim = [d,M,m,1]   #d-M-1
Err = 0


for times in range(t):
    w1 = initw(nndim[0],nndim[1])  
    w2 = initw(nndim[1],nndim[2])
    w3 = initw(nndim[2],nndim[3])

    for iteration in range(T):
        randomn = random.randint(0,N-1)
        x0 = np.insert(trainx[randomn],0,1)  #dimension of d+1
        x0 = np.reshape(x0,(-1,1))           #One shape dimension can be -1, inferred from the length of the array and remaining dim
        s1 = w1.T.dot(x0)
        x1 = np.tanh(s1)

        x1 = np.insert(x1,0,1,axis=0)  #dimension of 8+1
        s2 = w2.T.dot(x1)
        x2 = np.tanh(s2)
        
        x2 = np.insert(x2,0,1,axis=0)  #dimension of 3+1
        s3 = w3.T.dot(x2)
        x3 = np.tanh(s3)
        
        delta3 = (-8)*(trainy[randomn]-np.tanh(s3))/(np.exp(s3)+np.exp(-s3))
        delta2 = delta3*w3[1:]*4/(np.exp(s2)+np.exp(-s2))
        delta1 = w2[1:].dot(delta2)*4/(np.exp(s1)+np.exp(-s1))
        
        
        w3 = updatew(w3,x2,delta3)
        w2 = updatew(w2,x1,delta2)
        w1 = updatew(w1,x0,delta1)


    Err = Err+err(w1,w2,w3,testx,testy,testn)
#    print(err(w1,w2,w3,testx,testy,testn))

        
Eout = Err/t

end = time.time()
print('M =',M,'r =',r,'eta =',eta,'Eout =',Eout,'time consumed:',end-start)

0.064
0.044
0.056
0.032
0.044
0.048
0.044
0.056
0.048
0.052
0.052
0.048
0.044
0.048
0.036
0.032
0.056
0.044
0.052
0.044
0.04
0.072
0.056
0.04
0.056
0.068
0.036
0.056
0.052
0.044
0.032
0.056
0.052
0.036
0.056
0.06
0.044
0.048
0.052
0.052
0.068
0.056
0.036
0.044
0.032
0.048
0.048
0.032
0.036
0.048
0.048
0.052
0.052
0.068
0.048
0.04
0.044
0.052
0.048
0.052
0.056
0.056
0.036
0.048
0.04
0.032
0.068
0.036
0.044
0.04
0.048
0.052
0.036
0.048
0.04
0.048
0.036
0.052
0.068
0.048
0.036
0.056
0.04
0.064
0.06
0.036
0.048
0.052
0.04
0.044
0.044
0.044
0.048
0.06
0.044
0.032
0.044
0.044
0.056
0.044
0.044
0.04
0.068
0.056
0.04
0.056
0.044
0.056
0.052
0.056
0.048
0.044
0.04
0.044
0.044
0.036
0.056
0.044
0.048
0.048
0.056
0.04
0.068
0.044
0.052
0.044
0.036
0.056
0.04
0.04
0.044
0.044
0.036
0.04
0.048
0.056
0.036
0.036
0.036
0.06
0.048
0.056
0.056
0.044
0.064
0.032
0.064
0.048
0.036
0.048
0.04
0.052
0.056
0.064
0.048
0.044
0.036
0.036
0.04
0.048
0.048
0.036
0.04
0.04
0.068
0.044
0.044
0.036
0.048
0.044
0.0